In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LeakyReLU, BatchNormalization
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
data = pd.read_csv('./BSACTData.csv', index_col='Timestamp')
data = data.dropna()

In [3]:
data['block-gen-mu'] = 560
data['tx-gen-time'] = 26.66

In [4]:
data.head()

,mempool-size,transactions-per-second,median-confirmation-time,transaction-fees,n-transactions-per-block,block-gen-mu,tx-gen-time
Timestamp,,,,,,,
15-06-2016 00:00,21228973.0,2.433333,14.13,83.386147,1705.276316,560,26.66
16-06-2016 00:00,25873598.0,1.550000,9.66,82.296660,1797.785235,560,26.66
17-06-2016 00:00,26867610.0,2.350000,9.30,78.902095,1703.033557,560,26.66
18-06-2016 00:00,25255652.5,2.150000,7.75,70.578306,1489.348101,560,26.66
19-06-2016 00:00,7502073.5,2.250000,6.00,63.143491,1233.322034,560,26.66


In [5]:
data.shape

(1414, 7)

In [6]:
X = data.drop(columns=['median-confirmation-time', 'transaction-fees', 'n-transactions-per-block'])
Y = data.loc[:, ['transaction-fees']]

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [8]:
X_train.head()

,mempool-size,transactions-per-second,block-gen-mu,tx-gen-time
Timestamp,,,,
08-03-2018 00:00,21909306.5,2.233333,560,26.66
23-07-2018 00:00,157956.5,1.483333,560,26.66
14-10-2017 00:00,41604137.5,2.566667,560,26.66
02-08-2016 00:00,3247602.5,2.150000,560,26.66
17-11-2016 00:00,5396091.0,2.416667,560,26.66


In [9]:
from keras.models import load_model

In [17]:
import tensorflow as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)

tf.Session(config=tf.ConfigProto(gpu_options=gpu_options,allow_soft_placement=True))

In [10]:
medConfModel = load_model('weights/MedianConfTime.hfpy')
txFeeModel = load_model('weights/TransactionFees.hfpy')
numTxModel = load_model('weights/numTransactions.hfpy')

In [16]:
actX = pd.DataFrame([[125000, 0.037509377344336084, 560, 26.66]], columns=['mempool-size', 'transactions-per-second', 'block-gen-mu', 'tx-gen-time'])
actX

,mempool-size,transactions-per-second,block-gen-mu,tx-gen-time
0,125000,0.037509,560,26.66


In [17]:
medConfModel.predict(actX), txFeeModel.predict(actX), numTxModel.predict(actX)

(array([[4.8770304]], dtype=float32),
 array([[35.148914]], dtype=float32),
 array([[2127.887]], dtype=float32))

In [38]:
actX = pd.DataFrame([[41604137.5, 2.5666, 56, 26.66]], columns=['mempool-size', 'transactions-per-second', 'block-gen-mu', 'tx-gen-time'])
actX

,mempool-size,transactions-per-second,block-gen-mu,tx-gen-time
0,41604137.5,2.5666,56,26.66


In [39]:
medConfModel.predict(actX), txFeeModel.predict(actX), numTxModel.predict(actX)

(array([[8.533049]], dtype=float32),
 array([[152.72658]], dtype=float32),
 array([[511.16837]], dtype=float32))

In [37]:
from sklearn.metrics import r2_score

In [59]:
Y_pred = txFeeModel.predict(X_train)

In [48]:
r2_score(Y_train, Y_pred) # Transaction Fees

0.5874670328158076